# 📝 Exercise M6.01

The aim of this notebook is to investigate if we can tune the hyperparameters
of a bagging regressor and evaluate the gain obtained.

We will load the California housing dataset and split it into a training and a
testing set.

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

data, target = fetch_california_housing(as_frame=True, return_X_y=True)
target *= 100  # rescale the target in k$
data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=0, test_size=0.5
)

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">If you want a deeper overview regarding this dataset, you can refer to the
Appendix - Datasets description section at the end of this MOOC.</p>
</div>

Create a `BaggingRegressor` and provide a `DecisionTreeRegressor` to its
parameter `estimator`. Train the regressor and evaluate its generalization
performance on the testing set using the mean absolute error.

In [2]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

bagged_trees = BaggingRegressor(
    estimator=DecisionTreeRegressor(max_depth=3),
    n_estimators=100,
)

bagged_trees.fit(data_train, target_train)

predicted = bagged_trees.predict(data_test)

test_score = mean_absolute_error(target_test, predicted)

test_score

57.503279355014996

Now, create a `RandomizedSearchCV` instance using the previous model and tune
the important parameters of the bagging regressor. Find the best parameters
and check if you are able to find a set of parameters that improve the default
regressor still using the mean absolute error as a metric.

<div class="admonition tip alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Tip</p>
<p class="last">You can list the bagging regressor's parameters using the <tt class="docutils literal">get_params</tt> method.</p>
</div>

In [10]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
# bagged_trees.get_params()

distributions = {
    "n_estimators": [*range(1, 100, 10)],
    'max_samples': np.arange(0, 1, 0.1), #if float is the percentage of the size of X
    'max_features': [0.2, 0.8, 1.0],
    "estimator__max_depth": [*range(2, 10, 1)]
}

randomized_bagging = RandomizedSearchCV(bagged_trees, distributions, scoring="neg_mean_absolute_error", random_state=0)

randomized_bagging.fit(data_train, target_train)

test_score = - randomized_bagging.score(data_test, target_test) # - to make it positive

test_score


C:\Users\awben\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\awben\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\awben\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\awben\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_para

39.17629203118985